Nama     : **Muhamad Ilyas**               
Email    : **181240000831@unisnu.ac.id**   
Domisili : **Kabupaten Jepara, Jawa Tengah**

In [83]:
#import pandas dan membuat dataframe
import pandas as pd

df = pd.read_csv('bbc-news-data.csv', sep='\t')
df

,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...
...,...,...,...,...
2220,tech,397.txt,BT program to beat dialler scams,BT is introducing two initiatives to help bea...
2221,tech,398.txt,Spam e-mails tempt net shoppers,Computer users across the world continue to i...
2222,tech,399.txt,Be careful how you code,A new European directive could put software w...
2223,tech,400.txt,US cyber security chief resigns,The man making sure US computer networks are ...


In [84]:
#menghapus kolom yang tidak diperlukan
df = df.drop(columns=['filename','content'], axis=1, inplace=True)

In [86]:
#melihat file
df.info()

AttributeError: ignored

In [ ]:
#melakukan one-hot-encoding dan membuat dataframe baru
category = pd.get_dummies(df.category)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='category')
df_baru

AttributeError: ignored

In [ ]:
#menggunakan stopwords
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
print(stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'bo

In [ ]:
#menghilangkan tanda baca dan angka pada content
contents_clean = []
for content_clean in df_baru:
    for word in stopwords:
        token = " " + word + " "
        content_clean = content_clean.replace(token, " ")
    contents_clean.append(content_clean)

In [ ]:
import numpy as np
df_baru['content_clean'] = np.array(contents_clean)

ValueError: ignored

In [ ]:
#mengubah nilai-nilai dari dataframe ke dalam tipe data numpy array
keterangan = df_baru['body'].values
label = df_baru[['Excellent', 'Satisfactory', 'Good', 'Bad', 'Unsatisfactory']].values

In [ ]:
#data training dan data test
from sklearn.model_selection import train_test_split
keterangan_latih, keterangan_test, label_latih, label_test = train_test_split(keterangan, label, test_size=0.2)

In [ ]:
#mengubah dataset menjadi bilangan numerik
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences 

tokenizer = Tokenizer(num_words=10000, oov_token='x')
tokenizer.fit_on_texts(keterangan_latih)
tokenizer.fit_on_texts(keterangan_test)

sekuens_latih = tokenizer.texts_to_sequences(keterangan_latih)
sekuens_test = tokenizer.texts_to_sequences(keterangan_test)

padded_latih = pad_sequences(sekuens_latih)
padded_test = pad_sequences(sekuens_test)

AttributeError: ignored

In [ ]:
#arsitektur model kita kali mini menggunakan 3 buah layer.
import tensorflow as tf
from tensorflow.keras.layers import Dropout
model = tf.keras.Sequential([
      tf.keras.layers.Embedding(input_dim=10000, output_dim=16),
      tf.keras.layers.LSTM(64),
      tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(5, activation='softmax')
])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                20736     
_________________________________________________________________
dense_3 (Dense)              (None, 512)               33280     
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                

In [ ]:
#menentukan optimizer dan loss function dari model
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#memanggil Callback
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.95):
      print("\nTraining sudah mencapai akurasi diatas 95%")
      self.model.stop_training = True
callbacks = myCallback()

In [ ]:
#melatih model dengan fungsi fit
num_epochs=30
history = model.fit(padded_latih, label_latih, epochs=num_epochs,
                    validation_data=(padded_test, label_test),
                    verbose=2,
                    callbacks=[callbacks],
                    batch_size=256)

Epoch 1/30
65/65 - 233s - loss: 1.3375 - accuracy: 0.4348 - val_loss: 1.0962 - val_accuracy: 0.4794
Epoch 2/30
65/65 - 242s - loss: 0.9875 - accuracy: 0.5373 - val_loss: 0.9991 - val_accuracy: 0.5396
Epoch 3/30
65/65 - 239s - loss: 0.8396 - accuracy: 0.6158 - val_loss: 0.9548 - val_accuracy: 0.5638
Epoch 4/30
65/65 - 241s - loss: 0.7460 - accuracy: 0.6697 - val_loss: 0.9738 - val_accuracy: 0.5958
Epoch 5/30
65/65 - 242s - loss: 0.6575 - accuracy: 0.7153 - val_loss: 1.0457 - val_accuracy: 0.5545
Epoch 6/30
65/65 - 244s - loss: 0.6033 - accuracy: 0.7426 - val_loss: 1.1328 - val_accuracy: 0.5494
Epoch 7/30
65/65 - 234s - loss: 0.5260 - accuracy: 0.7833 - val_loss: 1.2087 - val_accuracy: 0.5526
Epoch 8/30
65/65 - 236s - loss: 0.4670 - accuracy: 0.8123 - val_loss: 1.4884 - val_accuracy: 0.5684
Epoch 9/30
65/65 - 242s - loss: 0.4179 - accuracy: 0.8371 - val_loss: 1.5418 - val_accuracy: 0.5555
Epoch 10/30
65/65 - 233s - loss: 0.3830 - accuracy: 0.8550 - val_loss: 1.6079 - val_accuracy: 0.5623